Musisz przygotować API działające po protokole HTTPS, a następnie wysłać do centrali URL do tego API jako odpowiedź na zadanie o nazwie webhook.

Pracownik centrali wyśle na Twoje API metodą POST dane w formacie JSON w formie jak poniżej:

{
"instruction":"tutaj instrukcja gdzie poleciał dron"
}


Opis lotu drona może być w dowolnej formie i jest to tekst w języku naturalnym, np. “poleciałem jedno pole w prawo, a później na sam dół”. Twoim zadaniem jest odpowiedzenie w maksymalnie dwóch słowach, co tam się znajduje. W naszym przykładzie odpowiedzią byłyby np. “skały”.

Centrala wyśle do Twojego API kilka takich opisów lotów. Jeden po drugim. Każdy w oddzielnym zapytaniu. Zadanie uznawane jest za zaliczone, gdy wszystkie loty skończą się sukcesem. Przekazanie pilotowi nieprawdziwych informacji nawigacyjnych kończy się rozbiciem drona.

W tym zadaniu możesz wykorzystać hosting na Azylu, jeśli masz na to ochotę (nie ma obowiązku tego robić!). Możesz także wykorzystać np. usługę ngrok, aby wystawić nam swoją lokalną aplikację wprost z Twojego dysku.

Jak wysłać URL do API do centrali?

{
 "apikey":"TWOJ-KLUCZ",
 "answer":"https://azyl-12345.ag3nts.org/moje_api",
 "task":"webhook"
}


Co należy zrobić w zadaniu?

Tworzysz aplikację (hostowaną tam, gdzie zechcesz), która przyjmuje requesty w formie JSON-a po HTTPS.

Musisz przygotować dość złożonego prompta, który w sposób zrozumiały dla LLM-a wyjaśni mu, jak wygląda mapa, którą właśnie oglądasz. Sugerujemy NIE korzystać tutaj z modelu graficznego rozpoznającego obraz. To mogłoby być bardzo kosztowne i skomplikowane.

Wysyłasz URL-a do Twojego API do centrali

Centrala wysyła serię zapytań pod adres URL, który został wysłany w polu “answer”

JSON od centrali zawiera pole “instruction” z opisem lotu drona

Twoim zadaniem jest zwrócenie poprawnego JSON-a z polem “description” zawierającego opis pola, na którym znajduje się dron po odbyciu lotu

Opis miejsca, nad którym zawisł dron może mieć maksymalnie dwa słowa - opis ma być po polsku

Twój JSON może mieć tyle pól, ile potrzebujesz (przydatne przy ‘głośnym myśleniu’!), ale tylko description jest brane pod uwagę.

Jeśli zaliczysz trzy loty, otrzymasz flagę. Flaga zostanie przesłana w odpowiedzi na wywołanie, w którym podajesz URL swojej aplikacji do centrali (punkt 3 instrukcji).

UWAGA: absolutnie kluczowym elementem zadania jest sprytne opisanie modelowi językowemu, jak wygląda świat z mapy. Musi to być zrobione w sposób jednoznaczny. Pamiętaj także, że mapa ma formę kwadratu 4x4. Pamiętaj także, że Twoje API powinno być bezstanowe, co oznacza, że NIE zapamiętujesz, gdzie skończył się ostatni lot drona. Zawsze zaczynasz z pozycji startowej.

Jeśli masz aktywne konto na Azylu, możesz też użyć SSH do przekierowania swojego lokalnego portu (na przykładzie = 3000) na swój port na Azylu (na przykladzie = 50005) w ten sposób:

ssh -R 50005:localhost:3000 agent10005@azyl.ag3nts.org -p 5022

Oczywiście podajesz swoją nazwę użytkownika. SSH po zalogowaniu będzie wyświetlało linię poleceń, ale będzie też działający tunel w tle, który przekieruje wywołania twojej dedykowanej strony (na przykładzie = https://azyl-50005.ag3nts.org/) do twojej aplikacji lokalnej.

UWAGA - przy takim rozwiązaniu, Twoja aplikacja powinna serwować ruch nieszyfrowany, czyli http:// i zostawić Azylowi jego zaszyfrowanie (dzieje sie to automatycznie)

In [18]:
import json
MAPA = {
    "[0,0]": "start",
    "[0,1]": "łąka",
    "[0,2]": "drzewo",
    "[0,3]": "dom",
    "[1,0]": "łąka",
    "[1,1]": "młyn",
    "[1,2]": "łaka",
    "[1,3]": "łaka",
    "[2,0]": "łąka",
    "[2,1]": "łąka",
    "[2,2]": "skały",
    "[2,3]": "drzewa",
    "[3,0]": "skały",
    "[3,1]": "skały",
    "[3,2]": "samochód",
    "[3,3]": "jaskinia"
}
print(json.dumps(MAPA, indent=4).encode('utf-8').decode('unicode_escape'))


{
    "[0,0]": "start",
    "[0,1]": "łąka",
    "[0,2]": "drzewo",
    "[0,3]": "dom",
    "[1,0]": "łąka",
    "[1,1]": "młyn",
    "[1,2]": "łaka",
    "[1,3]": "łaka",
    "[2,0]": "łąka",
    "[2,1]": "łąka",
    "[2,2]": "skały",
    "[2,3]": "drzewa",
    "[3,0]": "skały",
    "[3,1]": "skały",
    "[3,2]": "samochód",
    "[3,3]": "jaskinia"
}


In [19]:
import os
from dotenv import load_dotenv

load_dotenv()
personal_api_key = os.getenv("PERSONAL_API_KEY")


In [33]:
import requests
## Wysyłam api do centrali
url_api = {
 "apikey":personal_api_key,
 "answer":"https://azyl-51364.ag3nts.org",
 "task":"webhook"
}

answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=url_api)
print(answer_response.request.body)  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8').decode('unicode-escape'))       # Displays the response content as a string



b'{"apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": "https://azyl-51364.ag3nts.org", "task": "webhook"}'
Status Code: 200
Response Body: {
    "code": 0,
    "message": "{{FLG:DARKCAVE}}"
}


In [1]:
!sshpass -p 'SI4uIJbTW5' ssh -R 51364:localhost:3000 agent11364@azyl.ag3nts.org -p 5022

                 _              _____       _                        
  __ _ _____   _| |  __ _  __ _|___ / _ __ | |_ ___   ___  _ __ __ _ 
 / _` |_  / | | | | / _` |/ _` | |_ \| '_ \| __/ __| / _ \| '__/ _` |
| (_| |/ /| |_| | || (_| | (_| |___) | | | | |_\__ \| (_) | | | (_| |
 \__,_/___|\__, |_(_)__,_|\__, |____/|_| |_|\__|___(_)___/|_|  \__, |
           |___/          |___/                                |___/ 
---------------------------------------------------------------------
Last login: Mon Dec  2 19:01:31 2024 from 185.119.151.162
---------------------------------------------------------------------
Twoj numer portu to: 51364
---------------------------------------------------------------------
Spraw, aby Twoja aplikacja słuchała na tym porcie.
Aplikacja będzie dostępna np. pod https://azyl-51364.ag3nts.org
---------------------------------------------------------------------
Nie musisz używać 'bun install' - wszystkie paczki są w systemie!
----------------------------------